In [3]:
import findspark
findspark.init()
import pyspark.sql.functions as f
from pyspark.sql import SparkSession

# import pyspark
# sc = pyspark.SparkContext()
# conf = pyspark.SparkConf().setAll([('spark.executor.memory', '10g'), ('spark.executor.cores', '2'),
#     ('spark.cores.max', '2'), ('spark.driver.memory','5g'), ("spark.kryoserializer.buffer.max","1g"),
#     ("spark.sql.execution.arrow.pyspark.enabled", "true")])
# sc.stop()
# sc = pyspark.SparkContext(conf=conf)

ss = SparkSession.builder.getOrCreate()
ss.sparkContext.setLogLevel("ERROR")
ss.sparkContext.getConf().getAll()


22/07/30 16:25:29 WARN Utils: Your hostname, DESKTOP-9EJISJ5 resolves to a loopback address: 127.0.1.1; using 172.18.237.79 instead (on interface eth0)
22/07/30 16:25:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/30 16:25:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.submitTime', '1659191130614'),
 ('spark.executor.memory', '10g'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.cores', '2'),
 ('spark.driver.memory', '5g'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.kryoseria

In [4]:
DATA_REES46 = "../data/raw/rees46/_/"
DATA_RR = "../data/raw/retailrocket/_/"
events = ss.read.parquet(DATA_RR+"events", header=True, inferSchema=True, nanValue="null")

In [5]:
# n interactions
n_interactions = events.count()
n_transactions = events.where(f.col("purchase")==1).agg(f.countDistinct("user_session_id")).toPandas().values[0,0]
avg_revenue = events.where(f.col("purchase")==1).groupby("user_session_id")\
    .agg(f.sum(f.col("price")).alias("revenue")).agg(f.mean(f.col("revenue")).alias("avg_rev")).toPandas().values[0,0]
n_users = events.select("user_id").distinct().count()
n_customers = events.where(f.col("purchase")==1).select("user_id").distinct().count() 
# time range
events.agg(f.min(f.col("event_time")).alias("min_time"), f.max(f.col("event_time")).alias("max_time")).show()  

+--------------------+--------------------+
|            min_time|            max_time|
+--------------------+--------------------+
|2015-05-09 22:00:...|2015-09-17 21:59:...|
+--------------------+--------------------+

